<a href="https://colab.research.google.com/github/PyBeginner1/FraudulentFirmPrediction/blob/main/FraudulentFirmPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
data = pd.read_csv('/content/audit_data.csv')

In [11]:
data

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,23,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,1.0,3.38,0.2,0.676,2,0.2,0.4,0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1
1,3.89,6,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5.0,0.2,1.0,0.94,0.2,0.188,2,0.2,0.4,0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0
2,3.89,6,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0
3,3.89,6,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,3.6,11.75,0.6,7.050,2,0.2,0.4,0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1
4,3.89,6,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.416,0.4,0.5,0.2832,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,55.57,9,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156,0
772,55.57,16,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0
773,55.57,14,0.24,0.2,0.048,0.04,0.2,0.008,0.28,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0
774,55.57,18,0.20,0.2,0.040,0.00,0.2,0.000,0.20,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sector_score    776 non-null    float64
 1   LOCATION_ID     776 non-null    object 
 2   PARA_A          776 non-null    float64
 3   Score_A         776 non-null    float64
 4   Risk_A          776 non-null    float64
 5   PARA_B          776 non-null    float64
 6   Score_B         776 non-null    float64
 7   Risk_B          776 non-null    float64
 8   TOTAL           776 non-null    float64
 9   numbers         776 non-null    float64
 10  Score_B.1       776 non-null    float64
 11  Risk_C          776 non-null    float64
 12  Money_Value     775 non-null    float64
 13  Score_MV        776 non-null    float64
 14  Risk_D          776 non-null    float64
 15  District_Loss   776 non-null    int64  
 16  PROB            776 non-null    float64
 17  RiSk_E          776 non-null    flo

In [13]:
data.isnull().sum()

Sector_score      0
LOCATION_ID       0
PARA_A            0
Score_A           0
Risk_A            0
PARA_B            0
Score_B           0
Risk_B            0
TOTAL             0
numbers           0
Score_B.1         0
Risk_C            0
Money_Value       1
Score_MV          0
Risk_D            0
District_Loss     0
PROB              0
RiSk_E            0
History           0
Prob              0
Risk_F            0
Score             0
Inherent_Risk     0
CONTROL_RISK      0
Detection_Risk    0
Audit_Risk        0
Risk              0
dtype: int64

# Preprocessing

In [60]:
def preprocess_inputs(df):
  df = df.copy()

  #fill the missing value
  df['Money_Value'] = df['Money_Value'].fillna(df['Money_Value'].mean())

  #one hot encode location_id
  loc_dummies =  pd.get_dummies(df['LOCATION_ID'], prefix = 'location')
  df = pd.concat([df, loc_dummies], axis =1)
  df = df.drop(['LOCATION_ID'], axis = 1)

  #Split into features & Target
  X = df.drop(['Risk', 'Audit_Risk'], axis = 1)
  Y = df['Risk']

  # Train & test split
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True, random_state = 1)

  #Scaling
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)


  return X_train, X_test, Y_train, Y_test

In [61]:
X_train, X_test, Y_train, Y_test = preprocess_inputs(data)

In [62]:
X.shape,X_train.shape, X_test.shape


((776, 70), (543, 69), (233, 69))

In [63]:
Y

0      1
1      0
2      0
3      1
4      0
      ..
771    0
772    0
773    0
774    0
775    0
Name: Risk, Length: 776, dtype: int64

In [64]:
X_train

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,location_1,location_11,location_12,location_13,location_14,location_15,location_16,location_17,location_18,location_19,location_2,location_20,location_21,location_22,location_23,location_24,location_25,location_27,location_28,location_29,location_3,location_30,location_31,location_32,location_33,location_34,location_35,location_36,location_37,location_38,location_39,location_4,location_40,location_41,location_42,location_43,location_44,location_5,location_6,location_7,location_8,location_9,location_LOHARU,location_NUH,location_SAFIDON
11,-0.666896,2.392461,1.368968,2.402906,1.371985,1.657938,1.378710,1.753517,-0.232477,-0.278621,-0.266910,-0.202404,-0.582748,-0.206065,-0.399153,5.252651,1.070359,1.558508,2.660368,1.044634,1.510376,0.429590,1.461936,0.0,-0.096404,-0.150329,-0.237529,-0.233171,-0.174243,-0.210311,-0.285785,-0.042954,-0.129823,-0.300602,-0.233171,-0.096404,-0.105703,-0.185164,0.0,-0.042954,-0.074536,-0.105703,-0.105703,-0.162681,-0.060802,-0.042954,-0.129823,-0.205491,-0.042954,0.0,-0.042954,-0.060802,-0.105703,-0.074536,-0.096404,-0.233171,-0.074536,-0.042954,0.0,-0.096404,0.0,-0.254374,-0.224255,-0.060802,2.810985,-0.258452,-0.042954,0.0,-0.042954
214,-0.715284,0.588386,1.368968,0.618393,2.582421,1.657938,2.585408,2.449293,3.426516,4.938314,4.710943,0.222161,1.918693,0.228428,-0.399153,-0.169762,-0.412307,-0.197239,-0.257975,-0.173919,2.450359,1.062614,-0.389565,0.0,-0.096404,-0.150329,-0.237529,-0.233171,-0.174243,-0.210311,-0.285785,-0.042954,-0.129823,-0.300602,-0.233171,-0.096404,-0.105703,-0.185164,0.0,-0.042954,-0.074536,-0.105703,-0.105703,-0.162681,-0.060802,-0.042954,-0.129823,-0.205491,-0.042954,0.0,-0.042954,-0.060802,-0.105703,-0.074536,-0.096404,4.288689,-0.074536,-0.042954,0.0,-0.096404,0.0,-0.254374,-0.224255,-0.060802,-0.355747,-0.258452,-0.042954,0.0,-0.042954
88,-0.666896,0.810883,1.368968,0.838477,2.527320,1.657938,2.530477,2.448086,1.597019,2.329847,2.030561,-0.120626,0.667972,-0.146432,1.281003,-0.169762,1.070359,-0.197239,-0.257975,-0.173919,1.980368,0.728488,0.536185,0.0,-0.096404,-0.150329,-0.237529,-0.233171,-0.174243,-0.210311,-0.285785,-0.042954,-0.129823,-0.300602,-0.233171,-0.096404,-0.105703,-0.185164,0.0,-0.042954,-0.074536,-0.105703,-0.105703,-0.162681,-0.060802,-0.042954,-0.129823,4.866397,-0.042954,0.0,-0.042954,-0.060802,-0.105703,-0.074536,-0.096404,-0.233171,-0.074536,-0.042954,0.0,-0.096404,0.0,-0.254374,-0.224255,-0.060802,-0.355747,-0.258452,-0.042954,0.0,-0.042954
479,-0.751265,-0.420264,-0.904272,-0.404997,-0.426199,-0.677368,-0.413920,-0.472241,-0.232477,-0.278621,-0.266910,-0.217919,-0.582748,-0.211232,-0.399153,-0.169762,-0.412307,-0.197239,-0.257975,-0.173919,-0.839579,-0.336254,-0.389565,0.0,-0.096404,-0.150329,-0.237529,-0.233171,-0.174243,-0.210311,-0.285785,-0.042954,-0.129823,-0.300602,-0.233171,-0.096404,-0.105703,-0.185164,0.0,-0.042954,-0.074536,-0.105703,-0.105703,-0.162681,-0.060802,-0.042954,-0.129823,-0.205491,-0.042954,0.0,-0.042954,-0.060802,-0.105703,-0.074536,-0.096404,-0.233171,-0.074536,-0.042954,0.0,-0.096404,0.0,-0.254374,-0.224255,-0.060802,2.810985,-0.258452,-0.042954,0.0,-0.042954
602,1.470447,-0.347953,-0.904272,-0.381154,-0.405592,-0.677368,-0.407072,-0.438036,-0.232477,-0.278621,-0.266910,-0.217919,-0.582748,-0.211232,-0.399153,-0.169762,-0.412307,-0.197239,-0.257975,-0.173919,-0.839579,-0.332454,-0.389565,0.0,-0.096404,-0.150329,-0.237529,-0.233171,-0.174243,-0.210311,-0.285785,-0.042954,-0.129823,-0.300602,-0.233171,-0.096404,-0.105703,-0.185164,0.0,-0.042954,-0.074536,-0.105703,-0.105703,6.147009,-0.060802,-0.042954,-0.129823,-0.205491,-0.042954,0.0,-0.042954,-0.060802,-0.105703,-0.074536,-0.096404,-0.233171,-0.074536,-0.042954,0.0,-0.096404,0.0,-0.254374,-0.224255,-0.060802,-0.355747,-0.258452,-0.042954,0.0,-0.042954
...,...,...,

# Train the model 

In [65]:
models = {
    'Logistic regression' : LogisticRegression(),
    'KNeighbors Classifier' : KNeighborsClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'MLP Classifier' : MLPClassifier(),
    'Linear SVC' : LinearSVC(),
    'SVC' : SVC(),
    'Random Forest Classifier' : RandomForestClassifier(),
    'Gradient Boosting' : GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(X_train, Y_train)
  print(name + ' trained!!')

Logistic regression trained!!
KNeighbors Classifier trained!!
Decision Tree trained!!
MLP Classifier trained!!
Linear SVC trained!!
SVC trained!!
Random Forest Classifier trained!!
Gradient Boosting trained!!


In [66]:
for name, model in models.items():
  score = model.score(X_test, Y_test)
  print(name + 'score is :{:.2f}%'.format(score * 100))

Logistic regressionscore is :98.71%
KNeighbors Classifierscore is :90.13%
Decision Treescore is :100.00%
MLP Classifierscore is :99.57%
Linear SVCscore is :99.14%
SVCscore is :96.57%
Random Forest Classifierscore is :100.00%
Gradient Boostingscore is :99.57%


# Making Predictive System

In [71]:
input_data = (-0.666896,	2.392461,	1.368968,	2.402906,	1.371985,	1.657938,	1.378710,	1.753517,	-0.232477,-0.278621,	-0.266910,	-0.202404,	-0.582748,	-0.206065,	-0.399153,	5.252651,	1.070359,	1.558508,	2.660368,	1.044634,	1.510376,	0.429590,	1.461936	,0.0,	-0.096404,	-0.150329,	-0.237529	,-0.233171,	-0.174243,	-0.210311	,-0.285785,	-0.042954,	-0.129823,	-0.300602,	-0.233171,	-0.096404,-0.105703,	-0.185164,	0.0,	-0.042954,	-0.074536,	-0.105703,	-0.105703,	-0.162681,	-0.060802,	-0.042954,	-0.129823,	-0.205491,	-0.042954,	0.0,	-0.042954,	-0.060802,	-0.105703,	-0.074536,	-0.096404,	-0.233171,	-0.074536,	-0.042954,	0.0,	-0.096404,	0.0,	-0.254374,	-0.224255,	-0.060802,	2.810985,	-0.258452,	-0.042954,	0.0,	-0.042954)

input_numpy = np.asarray(input_data)

input_reshape = input_numpy.reshape(1, -1)

predict = model.predict(input_reshape)
if predict[0] == 1:
  print('Fraudlent Firm')
if predict[0] == 0:
  print('Non-Fraudulent Firm')

Fraudlent Firm
